## BEMM458 Final Assignment

## You are a data scientist working for the European Union. There is an increasing concern inn  relation to the spread and amplification of misinformation on social media.

## General management needs you to conduct an analysis of recent conversations posted on Twitter in relation to spain. You have to identify ongoing misinformation campaigns and profile those users contributing to the spread of misinformation.




In [1]:
import pandas as pd

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
pd.set_option('display.max_rows', 120)

In [4]:
pd.set_option('display.float_format', '{:.0f}'.format)

In [5]:
spainEnglishAll=pd.read_csv('./data/BEMM458FinalAssignment.csv')

In [6]:
pd.set_option('display.float_format', '{:.3f}'.format)

In [7]:
!pip install -U spacy==2.2.3

In [8]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 4.6 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [9]:
!pip install spacy_langdetect

In [10]:
!pip install spacy-readability

In [11]:
import spacy
from spacy import displacy
from spacy_readability import Readability
import spacy_langdetect
import en_core_web_sm


In [12]:
nlp = en_core_web_sm.load()

In [13]:
spainEnglishAll.columns

Index(['Unnamed: 0', 'tweeet.id', 'tweet.text', 'tweet.created_at',
       'tweet.author_id', 'tweet.lang', 'public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count'],
      dtype='object')

In [14]:
spainEnglishAll

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count
0,0,1461484086892146692,RT @ESPNFC: EURO 2008 top scorer: Villa ⚽️\nWo...,2021-11-18 23:58:54+00:00,3239744421.000,en,1568.000,0.000,0.000
1,1,1461484079245840390,RT @MrGeepGeep: SOE is over and now is the tim...,2021-11-18 23:58:52+00:00,2312771653.000,en,38.000,0.000,0.000
2,2,1461484043187462149,"RT @AnittaCrave: ""Envolver"" — by Anitta will d...",2021-11-18 23:58:43+00:00,1296857652677550080.000,en,90.000,0.000,0.000
3,3,1461483982105763842,"Men jewelry - Madrid, Spain \n@GuilleRBRB htt...",2021-11-18 23:58:29+00:00,1071703322166480768.000,en,0.000,0.000,5.000
4,4,1461483978658127872,"RT @Foreign_Cat: #Turkey &amp; #Spain, partner...",2021-11-18 23:58:28+00:00,1142741725225652224.000,en,34.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...
47545,47545,1461484434209783814,@only1womble @SimonJonesNews @pritipatel Someo...,2021-11-19 00:00:16+00:00,1733373504.000,en,0.000,0.000,0.000
47546,47546,1461484401712414722,RT @ancientorigins: A team of #scientists has ...,2021-11-19 00:00:09+00:00,100887413.000,en,18.000,0.000,0.000
47547,47547,1461484391255977994,On #Friday Amazing time in #SabatiniGardens #M...,2021-11-19 00:00:06+00:00,995441471938580352.000,en,0.000,0.000,1.000
47548,47548,1461484374558490625,RT @THEMAXHILTON: The wait is over ! you have...,2021-11-19 00:00:02+00:00,832015137263452160.000,en,121.000,0.000,0.000


In [15]:
spainEnglishAll=spainEnglishAll.dropna(subset=['tweet.author_id'])

In [16]:
spainEnglishAll=spainEnglishAll.drop(['Unnamed: 0'], axis=1)

## Question 1. Which twitter conversations have large levels of amplification ? (15 points)
### Tip: consider number of retweets, likes, etc as proxies to measure the spread of information on social media

In [17]:
Amplification= spainEnglishAll["public_metrics.retweet_count"]+spainEnglishAll["public_metrics.reply_count"]+spainEnglishAll["public_metrics.like_count"]
spainEnglishAll['Amplification_total']=Amplification
spainEnglishAll

,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,Amplification_total
0,1461484086892146692,RT @ESPNFC: EURO 2008 top scorer: Villa ⚽️\nWo...,2021-11-18 23:58:54+00:00,3239744421.000,en,1568.000,0.000,0.000,1568.000
1,1461484079245840390,RT @MrGeepGeep: SOE is over and now is the tim...,2021-11-18 23:58:52+00:00,2312771653.000,en,38.000,0.000,0.000,38.000
2,1461484043187462149,"RT @AnittaCrave: ""Envolver"" — by Anitta will d...",2021-11-18 23:58:43+00:00,1296857652677550080.000,en,90.000,0.000,0.000,90.000
3,1461483982105763842,"Men jewelry - Madrid, Spain \n@GuilleRBRB htt...",2021-11-18 23:58:29+00:00,1071703322166480768.000,en,0.000,0.000,5.000,5.000
4,1461483978658127872,"RT @Foreign_Cat: #Turkey &amp; #Spain, partner...",2021-11-18 23:58:28+00:00,1142741725225652224.000,en,34.000,0.000,0.000,34.000
...,...,...,...,...,...,...,...,...,...
47545,1461484434209783814,@only1womble @SimonJonesNews @pritipatel Someo...,2021-11-19 00:00:16+00:00,1733373504.000,en,0.000,0.000,0.000,0.000
47546,1461484401712414722,RT @ancientorigins: A team of #scientists has ...,2021-11-19 00:00:09+00:00,100887413.000,en,18.000,0.000,0.000,18.000
47547,1461484391255977994,On #Friday Amazing time in #SabatiniGardens #M...,2021-11-19 00:00:06+00:00,995441471938580352.000,en,0.000,0.000,1.000,1.000
47548,1461484374558490625,RT @THEMAXHILTON: The wait is over ! you have...,2021-11-19 00:00:02+00:00,832015137263452160.000,en,121.000,0.000,0.000,121.000


In [18]:
total_amp_count=spainEnglishAll.groupby('tweet.text').sum().sort_values(by=['public_metrics.retweet_count','public_metrics.reply_count','public_metrics.like_count'],ascending=False)

In [19]:
total_amp_count.head(50)

,tweet.author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,Amplification_total
tweet.text,,,,,
RT @ESPNFC: EURO 2008 top scorer: Villa ⚽️\nWorld Cup 2010 winner: Spain 🇪🇸\n\nEURO 2012 top scorer: Gomez ⚽️\nWorld Cup 2014 winner: Germany…,1076289296606272159744.000,2254784.000,0.000,0.000,2254784.000
"RT @jessehawken: The irony about ""superhero movies are the modern version of spaghetti westerns"" is that I would probably love superhero mo…",117367693413794627584.000,1739112.000,0.000,0.000,1739112.000
"RT @KatalinNovakMP: Meeting in Budapest with @Santi_ABASCAL, the leader of @vox_es, the largest right-wing party in Spain 🇪🇸. We strive to…",410322365921806188544.000,819636.000,0.000,0.000,819636.000
"RT @BintingM: Madrid, Spain 🇪🇸 #NaturePhotography #travelphotography #landscape #travel https://t.co/jVDiP3m0dY",542396026835837976576.000,650055.000,0.000,0.000,650055.000
"RT @NormOrnstein: It is not an exaggeration to say that Little Marco Rubio, by putting holds on key ambassadorships to China and Spain, is…",235281521718753165312.000,483172.000,0.000,0.000,483172.000
RT @kevin_boyx: Anyone in Madrid/ Spain wanna do some collaboration?! Please text me on DM ❤️ https://t.co/MG8Dq45gpm,392187233186315370496.000,464615.000,0.000,0.000,464615.000
"RT @loal_worldwide: ""What if Lionel Messi picked Spain over Argentina😳"" https://t.co/lgHniNyeKW",398453399473420697600.000,398275.000,0.000,0.000,398275.000
"RT @imbadatlife: In the last 20 years Spain has built 1,900 miles of high speed rail. The UK has built 67.",143876311876652187648.000,378350.000,0.000,0.000,378350.000
RT @viquirepublica: 📌This is the real problem in Catalonia .... Spain is a Fascist State \nHow long are you going to shut up Europe? https:/…,16455094756261271552.000,307179.000,0.000,0.000,307179.000


In [20]:
### While executing the code, A lot of tweets were repeating.Hence, I used the groupby function to remove all the duplicates and then added the number of retweets, likes, and replies associated to the tweet.

In [21]:
### The conversations which have the largest amount of amount of amplifications are:
###1.RT @ESPNFC: EURO 2008 top scorer: Villa ⚽️\nWorld Cup 2010 winner: Spain 🇪🇸\n\nEURO 2012 top scorer: Gomez ⚽️\nWorld Cup 2014 winner: Germany…
###2.RT @jessehawken: The irony about "superhero movies are the modern version of spaghetti westerns" is that I would probably love superhero mo…
###3.RT @KatalinNovakMP: Meeting in Budapest with @Santi_ABASCAL, the leader of @vox_es, the largest right-wing party in Spain 🇪🇸. We strive to…
###4.RT @BintingM: Madrid, Spain 🇪🇸 #NaturePhotography #travelphotography #landscape #travel https://t.co/jVDiP3m0dY
###5.RT @NormOrnstein: It is not an exaggeration to say that Little Marco Rubio, by putting holds on key ambassadorships to China and Spain, is…

## Question 2. Analysis of highly amplified conversations (15 points)

### Please describe  5 highly amplified conversations which are trying to polarize or misinform.

## Tip: given a tweet.id it is possible to have a detailed look at twitter by appending this id to the url: https://twitter.com/anyuser/status/


### For instance given the tweet.id: 1461577564850319362 you can inspect its content by having a look at: https://twitter.com/anyuser/status/1461577564850319362



In [22]:
total_amp_count.iloc[8]

tweet.author_id                16455094756261271552.000
public_metrics.retweet_count                 307179.000
public_metrics.reply_count                        0.000
public_metrics.like_count                         0.000
Amplification_total                          307179.000
Name: RT @viquirepublica: 📌This is the real problem in Catalonia .... Spain is a Fascist State \nHow long are you going to shut up Europe? https:/…, dtype: float64

In [23]:
viquirepublicaRT= "📌This is the real problem in Catalonia .... Spain is a Fascist State \nHow long are you going to shut up Europe?"

In [24]:
viquirepublicaRTProcessed=nlp(viquirepublicaRT)

In [25]:
displacy.render(viquirepublicaRTProcessed, style="ent")

In [26]:
## This tweet had negative comments and is polarizing.

In [27]:
total_amp_count.iloc[88]

tweet.author_id                2514036197856993280.000
public_metrics.retweet_count                  8022.000
public_metrics.reply_count                       0.000
public_metrics.like_count                        0.000
Amplification_total                           8022.000
Name: RT @GemsOfBollywood: "We ruled India for 1000 years. We ruled Spain for 800 years."\n- flaunts Pakistan superstar\n\nWhat he didn't say is thi…, dtype: float64

In [28]:
gems="We ruled India for 1000 years. We ruled Spain for 800 years.- flaunts Pakistan superstar What he didn't say is this: Now we are Bhikhari (beggar) nation. Our Mughal Badshah kids beg on railway platforms today. We don't have a word for science in Urdu"


In [29]:
gemsprocessed=nlp(gems)

In [30]:
displacy.render(gemsprocessed, style="ent")

In [31]:
## This tweet is misinofrming and has insulting comments againt a country.

In [32]:
total_amp_count.iloc[101]

tweet.author_id                1416473790775795712.000
public_metrics.retweet_count                  6073.000
public_metrics.reply_count                       0.000
public_metrics.like_count                        0.000
Amplification_total                           6073.000
Name: RT @MaziNnamdiKanu: Gerard Pique is a World Cup winner with over 103 caps for Spain. He never turned his back against Catalonia freedom. Se…, dtype: float64

In [33]:
mazi="Gerard Pique is a World Cup winner with over 103 caps for Spain. He never turned his back against Catalonia freedom. Severally, he has shown open support for Catalan freedom.But our Iheanacho & Ndidi with #Zoo flag?💔💔Shame!Difference btw White man & his Black counterparts."



In [34]:
maziprocessed=nlp(mazi)

In [35]:
displacy.render(maziprocessed, style="ent")

In [36]:
## This tweet is linking football with politics which has no connection. It was tweeted as political agenda and hence it is polarizing.

In [37]:
total_amp_count.iloc[123]

tweet.author_id                11527280251175026688.000
public_metrics.retweet_count                   3864.000
public_metrics.reply_count                        0.000
public_metrics.like_count                         0.000
Amplification_total                            3864.000
Name: RT @DefendAssange: For those who think legal cases are immune to political pressure, here's a US ambassador talking about how he and his co…, dtype: float64

In [38]:
assange="For those who think legal cases are immune to political pressure, here's a US ambassador talking about how he and his colleagues were pressuring Spain's judiciary to drop its investigation into US army killing of Spanish journalist Jose Couso in Iraq.https://wikileaks.org/plusd/cables/07MADRID911_a.html#efmCJqCfc"

In [39]:
assangeprocessed=nlp(assange)

In [40]:
displacy.render(assangeprocessed, style="ent")

In [41]:
## The user in this tweet has made some strong comments but the source is not credible.WikiLeaks cannot be credible so there is no way way of verfying the users comments.Hence, the information is misleading.

In [42]:
total_amp_count.iloc[207]

tweet.author_id                12384349814791245824.000
public_metrics.retweet_count                   1344.000
public_metrics.reply_count                        0.000
public_metrics.like_count                         0.000
Amplification_total                            1344.000
Name: RT @keewa: Recently at a Grand Tournament in Spain, a player calling himself "Austrian Painter" turned up in a Nazi branded top. He wasn't…, dtype: float64

In [43]:
keewa="Recently at a Grand Tournament in Spain, a player calling himself Austrian Painter turned up in a Nazi branded top. He wasn't expelled or censured in any way. Some players refused to play against him, and so the organisers called that a concession and gave him full points."

In [44]:
keewaprocessed=nlp(keewa)

In [45]:
displacy.render(keewaprocessed, style="ent")

In [46]:
## This tweet is racist and there is no way of veryfing this users tweet. Hence, the information is misleading.

In [47]:
## These 5 were selected from highly amplified conversations. I selected the first 5 tweets which i felt were misinforming or polarizing. Thier ranking is different as there were a lot of positive tweets in between.

## Question 3. Compute the z-score of the column "public_metrics.retweet_count" (15 points)
## tip: refer to the definition of Z-score at" https://www.statisticshowto.com/probability-and-statistics/z-score/

In [48]:
import pandas as pd
import numpy as np

In [49]:
new_df=pd.read_csv('./data/BEMM458FinalAssignment.csv')

In [50]:
new_df['z_score_retweet']=(new_df['public_metrics.retweet_count']-new_df['public_metrics.retweet_count'].mean())/new_df['public_metrics.retweet_count'].std()
new_df

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,z_score_retweet
0,0,1461484086892146692,RT @ESPNFC: EURO 2008 top scorer: Villa ⚽️\nWo...,2021-11-18 23:58:54+00:00,3239744421.000,en,1568.000,0.000,0.000,1.391
1,1,1461484079245840390,RT @MrGeepGeep: SOE is over and now is the tim...,2021-11-18 23:58:52+00:00,2312771653.000,en,38.000,0.000,0.000,-0.204
2,2,1461484043187462149,"RT @AnittaCrave: ""Envolver"" — by Anitta will d...",2021-11-18 23:58:43+00:00,1296857652677550080.000,en,90.000,0.000,0.000,-0.150
3,3,1461483982105763842,"Men jewelry - Madrid, Spain \n@GuilleRBRB htt...",2021-11-18 23:58:29+00:00,1071703322166480768.000,en,0.000,0.000,5.000,-0.244
4,4,1461483978658127872,"RT @Foreign_Cat: #Turkey &amp; #Spain, partner...",2021-11-18 23:58:28+00:00,1142741725225652224.000,en,34.000,0.000,0.000,-0.208
...,...,...,...,...,...,...,...,...,...,...
47545,47545,1461484434209783814,@only1womble @SimonJonesNews @pritipatel Someo...,2021-11-19 00:00:16+00:00,1733373504.000,en,0.000,0.000,0.000,-0.244
47546,47546,1461484401712414722,RT @ancientorigins: A team of #scientists has ...,2021-11-19 00:00:09+00:00,100887413.000,en,18.000,0.000,0.000,-0.225
47547,47547,1461484391255977994,On #Friday Amazing time in #SabatiniGardens #M...,2021-11-19 00:00:06+00:00,995441471938580352.000,en,0.000,0.000,1.000,-0.244
47548,47548,1461484374558490625,RT @THEMAXHILTON: The wait is over ! you have...,2021-11-19 00:00:02+00:00,832015137263452160.000,en,121.000,0.000,0.000,-0.117


In [51]:
### A higher zscore means the tweets were far more from the average number of retweets.

## Question 4. Filter conversations with a z-score larger than 10. (15 points)

In [52]:
new_df['z_score_rep']=(new_df['public_metrics.reply_count']-new_df['public_metrics.reply_count'].mean())/new_df['public_metrics.reply_count'].std()
new_df

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,z_score_retweet,z_score_rep
0,0,1461484086892146692,RT @ESPNFC: EURO 2008 top scorer: Villa ⚽️\nWo...,2021-11-18 23:58:54+00:00,3239744421.000,en,1568.000,0.000,0.000,1.391,-0.056
1,1,1461484079245840390,RT @MrGeepGeep: SOE is over and now is the tim...,2021-11-18 23:58:52+00:00,2312771653.000,en,38.000,0.000,0.000,-0.204,-0.056
2,2,1461484043187462149,"RT @AnittaCrave: ""Envolver"" — by Anitta will d...",2021-11-18 23:58:43+00:00,1296857652677550080.000,en,90.000,0.000,0.000,-0.150,-0.056
3,3,1461483982105763842,"Men jewelry - Madrid, Spain \n@GuilleRBRB htt...",2021-11-18 23:58:29+00:00,1071703322166480768.000,en,0.000,0.000,5.000,-0.244,-0.056
4,4,1461483978658127872,"RT @Foreign_Cat: #Turkey &amp; #Spain, partner...",2021-11-18 23:58:28+00:00,1142741725225652224.000,en,34.000,0.000,0.000,-0.208,-0.056
...,...,...,...,...,...,...,...,...,...,...,...
47545,47545,1461484434209783814,@only1womble @SimonJonesNews @pritipatel Someo...,2021-11-19 00:00:16+00:00,1733373504.000,en,0.000,0.000,0.000,-0.244,-0.056
47546,47546,1461484401712414722,RT @ancientorigins: A team of #scientists has ...,2021-11-19 00:00:09+00:00,100887413.000,en,18.000,0.000,0.000,-0.225,-0.056
47547,47547,1461484391255977994,On #Friday Amazing time in #SabatiniGardens #M...,2021-11-19 00:00:06+00:00,995441471938580352.000,en,0.000,0.000,1.000,-0.244,-0.056
47548,47548,1461484374558490625,RT @THEMAXHILTON: The wait is over ! you have...,2021-11-19 00:00:02+00:00,832015137263452160.000,en,121.000,0.000,0.000,-0.117,-0.056


In [53]:
new_df[new_df['z_score_rep']>10]

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,z_score_retweet,z_score_rep
2434,2434,1461450207657349131,EURO 2008 top scorer: Villa ⚽️\nWorld Cup 2010...,2021-11-18 21:44:16+00:00,18091004.000,en,1568.000,740.000,15423.000,1.391,161.377
6187,6187,1461403487384715266,Hi #Writers!\n\nHave you based any of your boo...,2021-11-18 18:38:37+00:00,1073111187917725568.000,en,8.000,60.000,104.000,-0.235,13.034
11438,11438,1461341020264742912,Another day of sky high electricity prices in ...,2021-11-18 14:30:24+00:00,1647573157.000,en,480.000,76.000,1011.000,0.257,16.524
11708,11708,1461338150610083843,"Madrid, Spain 🇪🇸 #NaturePhotogr...",2021-11-18 14:19:00+00:00,1383404685189545984.000,en,861.000,61.000,4240.000,0.654,13.252
14622,14622,1461307097816178688,We @FootballManager are currently looking for ...,2021-11-18 12:15:36+00:00,37889646.000,en,331.000,104.000,1025.000,0.101,22.632
15636,15636,1461295289608945670,"In the last 20 years Spain has built 1,900 mil...",2021-11-18 11:28:41+00:00,1662773784.000,en,658.000,57.000,2362.000,0.442,12.379
17547,17547,1461272444593860609,Why did Australia lose car manufacturing and w...,2021-11-18 09:57:54+00:00,826025269378969600.000,en,43.000,78.000,292.000,-0.199,16.960
20843,20843,1461225890226917377,Searching 100x #memecoin in this peace 😻\n\nOu...,2021-11-18 06:52:55+00:00,4593671721.000,en,401.000,57.000,529.000,0.174,12.379
27778,27778,1461806775594885122,From Spain: Manchester United are ready to bid...,2021-11-19 21:21:09+00:00,2430359540.000,en,379.000,512.000,6058.000,0.151,111.639
29129,29129,1461783899185549316,"In Oct 2021, Spain’s Minister of Health announ...",2021-11-19 19:50:14+00:00,1679362640.000,en,133.000,64.000,255.000,-0.105,13.906


In [54]:
new_df['z_score_retweet']=(new_df['public_metrics.retweet_count']-new_df['public_metrics.retweet_count'].mean())/new_df['public_metrics.retweet_count'].std()

In [55]:
new_df[new_df['z_score_retweet']>10]

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,z_score_retweet,z_score_rep
859,859,1461469510586839047,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 23:00:58+00:00,1036271517699587968.000,en,9909.000,0.000,0.000,10.084,-0.056
2252,2252,1461451876000489474,RT @MesutOzil1088: I grew up as a @Fenerbahce ...,2021-11-18 21:50:54+00:00,1245113611263057920.000,en,17801.000,0.000,0.000,18.309,-0.056
3009,3009,1461442074184720387,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 21:11:57+00:00,955569674737127424.000,en,9909.000,0.000,0.000,10.084,-0.056
4527,4527,1461421081105948679,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 19:48:32+00:00,1097937228607700992.000,en,9909.000,0.000,0.000,10.084,-0.056
4585,4585,1461420287510159373,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 19:45:23+00:00,1371813880343711488.000,en,9909.000,0.000,0.000,10.084,-0.056
5082,5082,1461415007846117376,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 19:24:24+00:00,2949011187.000,en,9909.000,0.000,0.000,10.084,-0.056
5403,5403,1461413368959492108,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 19:17:53+00:00,16095339.000,en,9909.000,0.000,0.000,10.084,-0.056
5484,5484,1461412419377537025,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 19:14:07+00:00,2385072439.000,en,9909.000,0.000,0.000,10.084,-0.056
5530,5530,1461411825971515395,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 19:11:45+00:00,984144046833721216.000,en,9909.000,0.000,0.000,10.084,-0.056
6012,6012,1461405606540103682,RT @viquirepublica: 📌This is the real problem ...,2021-11-18 18:47:02+00:00,2766567177.000,en,9909.000,0.000,0.000,10.084,-0.056


In [56]:
new_df['z_score_lik']=(new_df['public_metrics.like_count']-new_df['public_metrics.like_count'].mean())/new_df['public_metrics.like_count'].std()
new_df

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,z_score_retweet,z_score_rep,z_score_lik
0,0,1461484086892146692,RT @ESPNFC: EURO 2008 top scorer: Villa ⚽️\nWo...,2021-11-18 23:58:54+00:00,3239744421.000,en,1568.000,0.000,0.000,1.391,-0.056,-0.032
1,1,1461484079245840390,RT @MrGeepGeep: SOE is over and now is the tim...,2021-11-18 23:58:52+00:00,2312771653.000,en,38.000,0.000,0.000,-0.204,-0.056,-0.032
2,2,1461484043187462149,"RT @AnittaCrave: ""Envolver"" — by Anitta will d...",2021-11-18 23:58:43+00:00,1296857652677550080.000,en,90.000,0.000,0.000,-0.150,-0.056,-0.032
3,3,1461483982105763842,"Men jewelry - Madrid, Spain \n@GuilleRBRB htt...",2021-11-18 23:58:29+00:00,1071703322166480768.000,en,0.000,0.000,5.000,-0.244,-0.056,0.015
4,4,1461483978658127872,"RT @Foreign_Cat: #Turkey &amp; #Spain, partner...",2021-11-18 23:58:28+00:00,1142741725225652224.000,en,34.000,0.000,0.000,-0.208,-0.056,-0.032
...,...,...,...,...,...,...,...,...,...,...,...,...
47545,47545,1461484434209783814,@only1womble @SimonJonesNews @pritipatel Someo...,2021-11-19 00:00:16+00:00,1733373504.000,en,0.000,0.000,0.000,-0.244,-0.056,-0.032
47546,47546,1461484401712414722,RT @ancientorigins: A team of #scientists has ...,2021-11-19 00:00:09+00:00,100887413.000,en,18.000,0.000,0.000,-0.225,-0.056,-0.032
47547,47547,1461484391255977994,On #Friday Amazing time in #SabatiniGardens #M...,2021-11-19 00:00:06+00:00,995441471938580352.000,en,0.000,0.000,1.000,-0.244,-0.056,-0.023
47548,47548,1461484374558490625,RT @THEMAXHILTON: The wait is over ! you have...,2021-11-19 00:00:02+00:00,832015137263452160.000,en,121.000,0.000,0.000,-0.117,-0.056,-0.032


In [57]:
new_df[new_df['z_score_lik']>10]

,Unnamed: 0,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,z_score_retweet,z_score_rep,z_score_lik
314,314,1461479072219820034,"argentina, brasil and spain: last night in soh...",2021-11-18 23:38:58+00:00,1321957954371809280.000,en,142.000,13.000,2167.000,-0.096,2.780,20.378
2434,2434,1461450207657349131,EURO 2008 top scorer: Villa ⚽️\nWorld Cup 2010...,2021-11-18 21:44:16+00:00,18091004.000,en,1568.000,740.000,15423.000,1.391,161.377,145.231
7360,7360,1461388153638510597,Several foreign teams have already declared an...,2021-11-18 17:37:41+00:00,2430359540.000,en,88.000,16.000,1942.000,-0.152,3.435,18.259
11708,11708,1461338150610083843,"Madrid, Spain 🇪🇸 #NaturePhotogr...",2021-11-18 14:19:00+00:00,1383404685189545984.000,en,861.000,61.000,4240.000,0.654,13.252,39.903
15015,15015,1461302950232526860,@nocturnal_ly I went alone to Wembley day 1. I...,2021-11-18 11:59:07+00:00,1269231030290055168.000,en,32.000,2.000,1116.000,-0.210,0.381,10.479
15636,15636,1461295289608945670,"In the last 20 years Spain has built 1,900 mil...",2021-11-18 11:28:41+00:00,1662773784.000,en,658.000,57.000,2362.000,0.442,12.379,22.215
19850,19850,1461236789134376960,Xavi will begin to use a software that he used...,2021-11-18 07:36:13+00:00,885972982052597760.000,en,178.000,30.000,3651.000,-0.058,6.489,34.355
27778,27778,1461806775594885122,From Spain: Manchester United are ready to bid...,2021-11-19 21:21:09+00:00,2430359540.000,en,379.000,512.000,6058.000,0.151,111.639,57.026
32805,32805,1461735560435752961,"""What if Lionel Messi picked Spain over Argent...",2021-11-19 16:38:09+00:00,1443958261917511680.000,en,895.000,138.000,12216.000,0.689,30.050,115.025
33998,33998,1461720931664023556,"A UNESCO World Heritage site, the Alhambra pal...",2021-11-19 15:40:02+00:00,17471979.000,en,232.000,20.000,1670.000,-0.002,4.308,15.697


In [ ]:
### Selected reply,retweet and likes coloumn individually and calculated their z-score.

In [71]:
### Filtered the zscore of replies,retweet and likes coloumn individaully

## Question 5. Find the top 5 amplifiers (15 points)
### we consider amplifiers those twitter users who are involved in highly amplified conversations

In [72]:
top5=spainEnglishAll.groupby('tweet.author_id').sum()
top5.sort_values(by='Amplification_total', ascending=False).head()

,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,Amplification_total
tweet.author_id,,,,
1382234880465989632.000,277288.000,0.000,0.000,277288.000
1290910527393210368.000,127452.000,0.000,0.000,127452.000
1101831422631804800.000,105483.000,0.000,0.000,105483.000
1388490486181076992.000,57396.000,0.000,0.000,57396.000
1168957082.000,43945.000,0.000,0.000,43945.000


In [73]:
## These 5 users had the highest amplification total 

## Question 6. Extract, from the text colum, information on who created the conversation in the first place (15 points)
### Tip: retweeted conversations start with the pattern RT@
### For illustration purposes, the text "RT @20splentyforus: France, like UK should set a national 30km/h or 20mph urban default limit." was originally created by @20splentyforus
### Tip2. consider applying a regex pattern to the DataFrame using apply method

In [60]:
import re

In [61]:
retweets=spainEnglishAll['tweet.text'].str.contains('\S*RT',regex=True)

In [62]:
main=spainEnglishAll[retweets].drop_duplicates('tweet.text')

In [63]:
main['Conversation_creator']=main['tweet.text'].str.extract('((@)\w+)')[0]

In [70]:
main

,tweeet.id,tweet.text,tweet.created_at,tweet.author_id,tweet.lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,Amplification_total,Conversation_creator
0,1461484086892146692,RT @ESPNFC: EURO 2008 top scorer: Villa ⚽️\nWo...,2021-11-18 23:58:54+00:00,3239744421.000,en,1568.000,0.000,0.000,1568.000,@ESPNFC
1,1461484079245840390,RT @MrGeepGeep: SOE is over and now is the tim...,2021-11-18 23:58:52+00:00,2312771653.000,en,38.000,0.000,0.000,38.000,@MrGeepGeep
2,1461484043187462149,"RT @AnittaCrave: ""Envolver"" — by Anitta will d...",2021-11-18 23:58:43+00:00,1296857652677550080.000,en,90.000,0.000,0.000,90.000,@AnittaCrave
4,1461483978658127872,"RT @Foreign_Cat: #Turkey &amp; #Spain, partner...",2021-11-18 23:58:28+00:00,1142741725225652224.000,en,34.000,0.000,0.000,34.000,@Foreign_Cat
7,1461483906473988100,RT @KeithOlbermann: THE KIDS ARE ALLERGIC? SO ...,2021-11-18 23:58:11+00:00,1121666434893893632.000,en,112.000,0.000,0.000,112.000,@KeithOlbermann
...,...,...,...,...,...,...,...,...,...,...
47401,1461487357387939842,"RT @IBlackthorn: ""If you are looking for a goo...",2021-11-19 00:11:53+00:00,15435346.000,en,9.000,0.000,0.000,9.000,@IBlackthorn
47419,1461486851370500099,"RT @johnnywoodsx: Who’s in Spain (Madrid, Barc...",2021-11-19 00:09:53+00:00,197135135.000,en,5.000,0.000,0.000,5.000,@johnnywoodsx
47443,1461486438512533504,"RT @HerbertCHoover4: @NewDealChief nah, Franco...",2021-11-19 00:08:14+00:00,1448094110280212480.000,en,1.000,0.000,0.000,1.000,@HerbertCHoover4
47476,1461485781839749124,RT @Michael40199413: @stephenblanchar @mollygi...,2021-11-19 00:05:38+00:00,1451562785054330880.000,en,1.000,0.000,0.000,1.000,@Michael40199413


In [65]:
### regex code of \S*RT means the tweet should start with RT which takes into account the retweet
### regex code of @)\w+ means that it should take the character @ in tweets

## Question 7. Please briefly reflect on how would you try to mitigate misinformation on social media (2 paragraphs maximum). (10 points)

## Ans= I would make the people aware that the information they have shared is fake and request them to delete the post. I will also report such posts, which might alert the social media company to verify them and take them down. I would also post articles related to tackling misinformation on my social media feed so that my followers can read and educate others. I will also hold a workshop with the help of Exeter Media Watch Society in the university to make the students aware of sharing and trusting such misinformation and its drastic effects.

## Data scientists of social media companies can see the number of reports on posts and then verify if the post is fake or not. The post should be taken down if the post is reported by a large number of users. Social Media Companies can also reccommend their users to download extensions which will help them in veryfying the information.